In [2]:
import pandas as pd
import csv
import sqlite3
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import urllib
from scipy.stats import mannwhitneyu
import re
import os.path
from langdetect import detect
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer
import fastcluster
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
from scipy.spatial.distance import squareform
import sys
from scipy.cluster.hierarchy import fcluster, dendrogram
from nltk.stem.porter import PorterStemmer
import math
import gensim 
import hdbscan
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import warnings
warnings.filterwarnings('ignore')
sys.setrecursionlimit(100000)
matplotlib.style.use('ggplot')

# Cluster

In [32]:
videos = []
with open("sentences_en.tsv", 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        videos.append([row[0], row[1], row[2], row[3]])
        
videos_en_new = pd.DataFrame(videos, columns=['id', 'description', 'channelTitle', 'sentence'])
del videos

In [33]:
# videos_en_new = videos_en_new[:10000]

In [34]:
stemmer = PorterStemmer()

def tokenize(line):
    if (line is None):
        line = ''
    printable = set(string.printable)
    line = ''.join(filter(lambda x: x in printable, line)) 
    stopwords = nltk.corpus.stopwords.words('english')

    tokenizer = nltk.RegexpTokenizer(r'[a-zA-Z]*\'[a-zA-Z]*|\w+')
    
    tokens = []
    
    line = re.sub(r'(http[s]?://|www.)(?:[a-zA-Z]|[0-9]|[$-_@.&+]*|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))*', '', line).lower()
    tokens.extend(tokenizer.tokenize(line))
    
    tokens_ = [f.strip(string.punctuation) for f in tokens]
    tokens_ = [f for f in tokens_ if f != '' and f not in stopwords and len(f) != 1]
    tokens_ = [f for f in tokens_ if not (f.isdigit() or f[0] == '-' and f[1:].isdigit())]
    tokens_ = [stemmer.stem(f) for f in tokens_]

    return tokens_


In [35]:
countVec = CountVectorizer(tokenizer=tokenize).fit(videos_en_new['sentence'])
#try with binary=True as well

In [36]:
lineVec = countVec.transform(videos_en_new['sentence'])

In [37]:
print lineVec.shape

(155345, 68515)


In [38]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)

In [ ]:
clusterer.fit(lineVec)

In [ ]:
videos_en_new["cluster"] = clusterer.labels_